In [ ]:
# Cell 1. Импорт библиотек и базовые настройки

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)

print("Библиотеки загружены.")


In [ ]:
# Cell 2. Загрузка данных

file_path = 'РД_МНН до мая.xlsx'  # при необходимости поменяйте путь
data = pd.read_excel(file_path)

print("Размер исходных данных:", data.shape)
print("Первые строки:")
display(data.head())

print("\nСписок столбцов:", data.columns.tolist())
print("\nУникальных DFU:", data['DFU'].nunique())


In [ ]:
# Cell 3. Преобразование даты и сортировка

data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(['DFU', 'date']).reset_index(drop=True)

# Проверим один DFU для примера
example_dfu = data['DFU'].iloc[0]
print("Пример DFU:", example_dfu)

display(
    data[data['DFU'] == example_dfu].head(10)
)


In [ ]:
# Cell 4. Вспомогательные функции

def print_metrics(y_true, y_pred, name=""):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"{name}  MAE: {mae:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}")
    return mae, rmse, mape


def make_lag_rolling_features(df, target_col='sales', max_lag=7, rolling_window=7):
    """
    df: DataFrame с колонками ['date', target_col]
    Возвращает DataFrame с фичами, индекс = date.
    
    ВАЖНО: чтобы не было утечки информации, роллинг считаем по прошлым значениям:
    rolling_* = sales.shift(1).rolling(...)
    """
    work = df.copy().sort_values('date')
    work = work.set_index('date')

    # Лаги
    for i in range(1, max_lag + 1):
        work[f'lag_{i}'] = work[target_col].shift(i)

    # Роллинги (по прошлым значениям)
    work['rolling_mean'] = work[target_col].shift(1).rolling(window=rolling_window).mean()
    work['rolling_max']  = work[target_col].shift(1).rolling(window=rolling_window).max()
    work['rolling_min']  = work[target_col].shift(1).rolling(window=rolling_window).min()

    work = work.dropna()
    return work  # индекс - дата, есть колонка target_col и фичи


def monthly_aggregate_from_series(true_series, pred_series):
    """
    Принимает 2 Series с индексом-датой (любая частота: D/W/M).
    Возвращает два Series, агрегированных до месяцев (суммирование).
    Индекс - первое число месяца.
    """
    true_m = true_series.copy()
    pred_m = pred_series.copy()

    # Группируем по месяцу через period('M')
    true_m = true_m.groupby(true_m.index.to_period('M')).sum()
    pred_m = pred_m.groupby(pred_m.index.to_period('M')).sum()

    # Приведём к таймстэмпу первого дня месяца
    true_m.index = true_m.index.to_timestamp()
    pred_m.index = pred_m.index.to_timestamp()

    return true_m, pred_m


# Быстрая проверка make_lag_rolling_features на одном DFU (daily)
df_example = data[data['DFU'] == example_dfu][['date', 'sales']]
features_example = make_lag_rolling_features(df_example, max_lag=7, rolling_window=7)

print("Размер features_example:", features_example.shape)
display(features_example.head())


In [ ]:
# Cell 5. Обучение и прогноз для одной временной серии

def train_validate_forecast_one_series(
    df, 
    target_col='sales',
    max_lag=7,
    rolling_window=7,
    val_periods=60,
    horizon_periods=60,
    k_best_features=5
):
    """
    df: DataFrame с колонками ['date', target_col]
    max_lag, rolling_window: гиперпараметры фичей
    val_periods: кол-во последних периодов под валидацию
    horizon_periods: сколько периодов прогнозировать вперёд
    """

    # 1. Фичи
    feat_df = make_lag_rolling_features(df, target_col=target_col, max_lag=max_lag, rolling_window=rolling_window)
    if len(feat_df) < val_periods + 30:  # мало данных
        return None

    # 2. Train/Val сплит по последним val_periods
    last_idx = len(feat_df) - 1
    val_start_idx = last_idx - val_periods + 1

    train_df = feat_df.iloc[:val_start_idx]
    val_df   = feat_df.iloc[val_start_idx:]

    X_train = train_df.drop(columns=[target_col])
    y_train = train_df[target_col]
    X_val   = val_df.drop(columns=[target_col])
    y_val   = val_df[target_col]

    # 3. Отбор фич
    if k_best_features is not None and k_best_features < X_train.shape[1]:
        selector = SelectKBest(score_func=f_regression, k=k_best_features)
        selector.fit(X_train, y_train)
        best_cols = list(X_train.columns[selector.get_support()])
    else:
        best_cols = list(X_train.columns)

    X_train_sel = X_train[best_cols]
    X_val_sel   = X_val[best_cols]

    # 4. Модель
    model = LinearRegression()
    model.fit(X_train_sel, y_train)

    # 5. Валидация
    val_pred = model.predict(X_val_sel)
    mae, rmse, mape = print_metrics(y_val, val_pred, name="Валидация")

    # 6. Прогноз вперёд итеративно
    # исходная серия (по всей истории до конца VAL)
    series_all = df.copy().sort_values('date').set_index('date')[target_col]

    # Убедимся, что индекс соотносится с feat_df
    # (feat_df уже отбросил первые max_lag/роллинг)
    last_date = series_all.index.max()
    future_index = []

    # определим тип шага по разнице последних дат
    if len(series_all) >= 2:
        step = (series_all.index[-1] - series_all.index[-2])
    else:
        # fallback на дни
        step = pd.Timedelta(days=1)

    for i in range(1, horizon_periods + 1):
        future_index.append(last_date + i * step)

    future_preds = []
    current_series = series_all.copy()

    for new_date in future_index:
        # временный ряд с NaN на новой дате
        temp = current_series.copy()
        temp.loc[new_date] = np.nan

        # соберём фичи "вручную"
        feat_dict = {}
        # лаги
        for i in range(1, max_lag + 1):
            feat_dict[f'lag_{i}'] = temp.shift(i).loc[new_date]

        # роллинги
        temp_shifted = temp.shift(1)
        feat_dict['rolling_mean'] = temp_shifted.rolling(window=rolling_window).mean().loc[new_date]
        feat_dict['rolling_max']  = temp_shifted.rolling(window=rolling_window).max().loc[new_date]
        feat_dict['rolling_min']  = temp_shifted.rolling(window=rolling_window).min().loc[new_date]

        X_new = pd.DataFrame([feat_dict])[best_cols]
        y_new_pred = model.predict(X_new)[0]

        # сохраняем предсказание и обновляем ряд
        future_preds.append(y_new_pred)
        current_series.loc[new_date] = y_new_pred

    # Преобразуем в Series
    val_pred_series = pd.Series(val_pred, index=y_val.index, name='pred')
    val_true_series = y_val.copy()

    future_series = pd.Series(future_preds, index=pd.DatetimeIndex(future_index), name='forecast')

    return {
        'model': model,
        'best_cols': best_cols,
        'val_true': val_true_series,
        'val_pred': val_pred_series,
        'metrics': {'mae': mae, 'rmse': rmse, 'mape': mape},
        'future_forecast': future_series
    }


# Проверка функции train_validate_forecast_one_series на одном DFU (дни)

df_example = data[data['DFU'] == example_dfu][['date', 'sales']]
res_example = train_validate_forecast_one_series(
    df_example,
    max_lag=7,
    rolling_window=7,
    val_periods=60,
    horizon_periods=60,
    k_best_features=5
)

if res_example is not None:
    print("Валидация - длина:", len(res_example['val_true']))
    print("Прогноз будущего - длина:", len(res_example['future_forecast']))
    display(res_example['val_true'].tail())
    display(res_example['future_forecast'].head())
else:
    print("Недостаточно данных для примера.")


In [ ]:
# Cell 6. Кандидаты конфигураций лагов и роллингов, частоты

lag_rolling_configs = [
    {'max_lag': 7,  'rolling_window': 7},
    {'max_lag': 14, 'rolling_window': 7},
    {'max_lag': 14, 'rolling_window': 14},
]

freq_settings = {
    'D': {  # дни
        'resample_rule': None,  # не ресэмплим, уже дневные
        'val_periods': 60,
        'horizon_periods': 60
    },
    'W': {  # недели
        'resample_rule': 'W-MON',  # или другая нужная вам неделя
        'val_periods': int(np.ceil(60 / 7)),   # эквивалент 60 дней
        'horizon_periods': int(np.ceil(60 / 7))
    },
    'M': {  # месяцы
        'resample_rule': 'MS',  # начало месяца
        'val_periods': 2,       # примерно 60 дней ~ 2 месяца
        'horizon_periods': 2
    }
}

print("Конфигурации лагов/роллинга:", lag_rolling_configs)
print("Частоты:", freq_settings)


In [ ]:
# Cell 7. Подготовка ряда под нужную частоту (D/W/M)

def prepare_series_for_freq(df_dfu, freq_key):
    """
    df_dfu: данные по одному DFU (колонки ['date','sales'])
    freq_key: 'D', 'W', 'M'
    Возвращает DataFrame ['date','sales'] в требуемой частоте.
    """
    settings = freq_settings[freq_key]
    rule = settings['resample_rule']

    if rule is None:  # 'D'
        work = df_dfu[['date', 'sales']].copy()
        work = work.sort_values('date')
        # убедимся, что календарь полный (если нужно, можно reindex по полному диапазону)
        # здесь предполагаем, что пропуски = 0 уже есть в данных
        return work

    # агрегация с помощью resample
    tmp = df_dfu[['date', 'sales']].copy().set_index('date').sort_index()
    work = tmp.resample(rule).sum().reset_index()
    work = work.rename(columns={'date': 'date'})
    return work


# Проверка на одном DFU
df_example = data[data['DFU'] == example_dfu][['date', 'sales']]

for f in ['D', 'W', 'M']:
    tmp = prepare_series_for_freq(df_example, f)
    print(f"\nЧастота {f}, размер:", tmp.shape)
    display(tmp.head())


In [ ]:
# Cell 8. Основной цикл по DFU

all_results_rows = []   # сюда сложим итоговую таблицу
summary_best_models = []  # сводка по лучшим моделям (DFU, freq, конфиг, mape)

dfus = data['DFU'].unique()
print("Всего DFU:", len(dfus))

for dfu in dfus:
    df_dfu = data[data['DFU'] == dfu][['date', 'sales']].copy()
    df_dfu = df_dfu.sort_values('date')

    print(f"\n================ DFU: {dfu} ================")

    best_config_info = None  # сюда положим лучшую по MAPE модель
    best_monthly_val_true = None
    best_monthly_val_pred = None
    best_monthly_forecast = None

    for freq_key in ['D', 'W', 'M']:
        freq_conf = freq_settings[freq_key]
        series_freq = prepare_series_for_freq(df_dfu, freq_key)

        print(f"\n--- Частота: {freq_key}, размер ряда: {len(series_freq)} ---")

        for cfg in lag_rolling_configs:
            print(f"\nКонфигурация: max_lag={cfg['max_lag']}, rolling={cfg['rolling_window']}")
            res = train_validate_forecast_one_series(
                series_freq,
                target_col='sales',
                max_lag=cfg['max_lag'],
                rolling_window=cfg['rolling_window'],
                val_periods=freq_conf['val_periods'],
                horizon_periods=freq_conf['horizon_periods'],
                k_best_features=5
            )

            if res is None:
                print("  Недостаточно данных для этой конфигурации.")
                continue

            # Агрегация валидации до месяцев
            val_true = res['val_true']
            val_pred = res['val_pred']

            # индексы уже даты (из make_lag_rolling_features -> index=date)
            true_m, pred_m = monthly_aggregate_from_series(val_true, val_pred)

            if len(true_m) == 0:
                print("  После агрегации по месяцам данных нет.")
                continue

            mape_monthly = mean_absolute_percentage_error(true_m, pred_m)
            print(f"  Месячный MAPE (валидация): {mape_monthly:.4f}")

            # Прогноз вперёд и тоже в месяцы
            future_series = res['future_forecast']
            # future_series индекс — даты периода той же частоты
            # Нам нужны месячные суммы прогноза
            future_m_true_dummy = pd.Series(np.zeros_like(future_series.values), index=future_series.index)
            _, forecast_m = monthly_aggregate_from_series(future_m_true_dummy, future_series)

            # Обновляем лучшую конфигурацию
            if (best_config_info is None) or (mape_monthly < best_config_info['mape_monthly']):
                best_config_info = {
                    'dfu': dfu,
                    'freq_key': freq_key,
                    'max_lag': cfg['max_lag'],
                    'rolling_window': cfg['rolling_window'],
                    'mape_monthly': mape_monthly
                }
                best_monthly_val_true = true_m
                best_monthly_val_pred = pred_m
                best_monthly_forecast = forecast_m

    if best_config_info is None:
        print(f"Для DFU {dfu} не удалось подобрать модель (слишком мало данных).")
        continue

    print("\nЛучшая модель для DFU:", dfu)
    print(best_config_info)

    summary_best_models.append(best_config_info)

    # Сохраним строки итоговой таблицы: валидация (месячная)
    for month, actual_val in best_monthly_val_true.items():
        pred_val = best_monthly_val_pred.loc[month] if month in best_monthly_val_pred.index else np.nan
        row = {
            'DFU': dfu,
            'best_freq': best_config_info['freq_key'],
            'max_lag': best_config_info['max_lag'],
            'rolling_window': best_config_info['rolling_window'],
            'segment': 'validation',
            'month': month,
            'actual_sales_month': actual_val,
            'pred_sales_month': pred_val,
        }
        # MAPE на уровне месяца (по одному месяцу можно считать абсолютное относительное отклонение)
        if actual_val != 0:
            row['abs_perc_error'] = abs(actual_val - pred_val) / abs(actual_val)
        else:
            row['abs_perc_error'] = np.nan
        all_results_rows.append(row)

    # Сохраним строки итоговой таблицы: прогноз (месячный)
    for month, forecast_val in best_monthly_forecast.items():
        row = {
            'DFU': dfu,
            'best_freq': best_config_info['freq_key'],
            'max_lag': best_config_info['max_lag'],
            'rolling_window': best_config_info['rolling_window'],
            'segment': 'forecast',
            'month': month,
            'actual_sales_month': np.nan,  # будущие факты неизвестны
            'pred_sales_month': forecast_val,
            'abs_perc_error': np.nan
        }
        all_results_rows.append(row)


In [ ]:
# Cell 9. Формирование итоговой таблицы и сохранение

results_df = pd.DataFrame(all_results_rows)
results_df = results_df.sort_values(['DFU', 'segment', 'month']).reset_index(drop=True)

print("Размер итоговой таблицы:", results_df.shape)
display(results_df.head(20))

# Сводка по лучшим моделям
summary_df = pd.DataFrame(summary_best_models)
print("\nСводка по лучшим конфигурациям для каждого DFU:")
display(summary_df)

# Сохранение в Excel
output_path_results = 'final_results_monthly_validation_and_forecast.xlsx'
output_path_summary = 'best_models_summary.xlsx'

results_df.to_excel(output_path_results, index=False)
summary_df.to_excel(output_path_summary, index=False)

print("\nФайлы сохранены:")
print(" -", output_path_results)
print(" -", output_path_summary)
